# newm toplevel in notebook form
There are two ways to expose a running python app to jupyter REPL access.
1. Embed a kernel: start serving the jupyter kernel protocol from the app.  See [this blog](https://blog.jupyter.org/plug-your-application-into-the-jupyter-world-805e48918801).
2. Launch the app from jupyter.  Perhaps simpler — "just" take the top level script launching your app and convert it into notebook form.  <-- DOING THIS HERE

In [1]:
x

NameError: name 'x' is not defined

In [1]:
x = 43 # to test concurrent connections

In [2]:
from __future__ import annotations
from typing import Optional
import io
import os, os.path
import sys
import threading
import time
import logging

In [3]:
%load_ext autoreload

In [4]:
%autoreload 3
import newm.layout, newm.config
%aimport

ModuleNotFoundError: No module named 'pywm'

## Logging & config

In [8]:
!which pip

/home/beni/WM/jupy/bin/pip


In [20]:
# https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html#interacting-with-output-widgets-from-background-threads
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Asynchronous.html#updating-a-widget-in-the-background
import threading
from IPython.display import display, HTML
import ipywidgets as widgets
import time

def thread_func():
    total = 60
    for i in range(0, total):
           
        time.sleep(1)
        progress.value = float(i+1)/total
        if i % 5 == 4:
            #out.append_stderr(f'Buzz{i} ')
            # Prepend?
            out.outputs = ({'name': 'stderr', 'output_type': 'stream', 'text': (f'Buzz{i} ')},) + out.outputs
        else:
            #print(f'{i}', end=' ')
            out.append_stdout(f'{i} ')

        # Don't interfere if this cell got re-evaluated and a new thread started
        if threading.current_thread() is not thread:
            break
        
    out.append_display_data(HTML(f"<em>All done! {threading.current_thread()}</em>"))

progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
out = widgets.Output()
thread = threading.Thread(target=thread_func, name='widgets test')
thread.start()


In [21]:
# Now the key: the container is displayed (while empty) in the main thread
# * After browser close & r, you may need to re-display the widget objects.
#   That will allow you to see live updates, including past outputs, from still-running thread!
display('Display in main thread')
progress

'Display in main thread'

FloatProgress(value=0.0, max=1.0)

In [22]:
display(out)

# `Output.__enter__` does not replace sys.stdout; it remains same `ipykernel.iostream.OutStream` instance.
# Instead it mucks something about msg_id - to route 'stream' messages to the widget I guess?
with out:
    for j in range(100, 105):
        print(j)

Output()

In [23]:
threading.enumerate()

[<_MainThread(MainThread, started 140211985966912)>,
 <Thread(IOPub, started daemon 140211681253056)>,
 <Heartbeat(Heartbeat, started daemon 140211672860352)>,
 <Thread(Thread-2 (_watch_pipe_fd), started daemon 140211647682240)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 140211639289536)>,
 <ControlThread(Control, started daemon 140211556185792)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140211547793088)>,
 <ParentPollerUnix(Thread-1, started daemon 140211539400384)>,
 <Thread(widgets test, started 140211525764800)>]

### Does IPykernel capture C-level writes to fd 1, 2?

In [ ]:
import sys
sys.__stderr__.write('='*50 + '\n')

51

In [41]:
import os
os.system('echo /////////// 1>&2')

///////////


0

In [56]:
# That works via pipes:
!ls -lv /proc/{os.getpid()}/fd | grep pipe

lr-x------ 1 beni beni 64 Jan 20 11:24 0 -> pipe:[13940338]
l-wx------ 1 beni beni 64 Jan 20 11:24 1 -> pipe:[13947597]
l-wx------ 1 beni beni 64 Jan 20 11:24 2 -> pipe:[13947598]
lr-x------ 1 beni beni 64 Jan 20 11:24 36 -> pipe:[13947597]
l-wx------ 1 beni beni 64 Jan 20 11:24 37 -> pipe:[13947597]
lr-x------ 1 beni beni 64 Jan 20 11:24 39 -> pipe:[13947598]
l-wx------ 1 beni beni 64 Jan 20 11:24 40 -> pipe:[13947598]
lr-x------ 1 beni beni 64 Jan 20 11:27 60 -> pipe:[14983695]
l-wx------ 1 beni beni 64 Jan 20 11:27 62 -> pipe:[14983695]


In [25]:
# These params mirror arguments of `def run(debug...)` in newm/run.py
debug: bool=True
#profile: bool=False
config_file: Optional[str]= os.path.expanduser('~/.config/newm/config.py')
log_file: str= os.path.expanduser('~/.cache/newm/newm_log')

NameError: name 'os' is not defined

In [16]:
logger = logging.getLogger(__name__)
handler = logging.FileHandler(log_file)

if debug:
    formatter = logging.Formatter('[%(levelname)s] %(filename)s:%(lineno)s %(asctime)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    handler.setLevel(logging.DEBUG)
else:
    formatter = logging.Formatter('[%(levelname)s] %(asctime)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    handler.setLevel(logging.INFO)

handler.setFormatter(formatter)

for l in ["newm", "pywm"]:
    log = logging.getLogger(l)
    log.setLevel(logging.DEBUG)
    log.addHandler(handler)

NameError: name 'log_file' is not defined

- [ ] TODO: pywm should give `wlr_log_init()` a callback to control logging destination?


## Redirect C-level stdout from pywm->wlroots as well.
(For more details see https://eli.thegreenplace.net/2015/redirecting-all-kinds-of-stdout-in-python/ and https://github.com/minrk/wurlitzer package.)

In [1]:
try:
    log_stream  # idempotent
except NameError:
    log_stream = open(log_file, 'a')

# Make file descriptor 1 (stdout) and 2 (stderr) point to the same file.
os.dup2(log_stream.fileno(), 1)
os.dup2(log_stream.fileno(), 2)
# Do NOT touch Python-level sys.stdout/err which are ipykernel proxies.
# Since we run wlroots in background thread(s), don't bother restoring :-p

print('@@@ test Python-level stdout still goes to notebook')
os.system('echo @@@ test C-level redirection')

NameError: name 'log_file' is not defined

In [9]:
logging.root.getChildren()

{<Logger Comm (WARNING)>,
 <Logger IPKernelApp (DEBUG)>,
 <Logger __main__ (WARNING)>,
 <Logger asyncio (WARNING)>,
 <Logger autoreload (WARNING)>,
 <Logger newm (DEBUG)>,
 <Logger parso (WARNING)>,
 <Logger pywm (DEBUG)>,
 <Logger tornado (WARNING)>}

## Run in a thread — so cell "finishes" and we can explore _while still running_

* 🤔 try using jupyter debugging, which automatically [runs in separate thread](https://www.romaglushko.com/blog/jupyter-kernel-architecture/#code-debugging) ?
* 🎉 [JEP91 subshells](https://github.com/jupyter/enhancement-proposals/blob/master/kernel-subshells/kernel-subshells.md) added protocol for executing in separate threads, coming [to ipykernel](https://github.com/ipython/ipykernel/pull/1249) and [to jupyterlab](https://github.com/jupyterlab/jupyterlab/issues/16882).
* https://github.com/jupyter/jupyter_client/issues/797 tracks background execution of cells; they're undecided whether to achieve it via subshells or else...

In [10]:
try:
    # Allow "Run all cells" again when newm is already running
    wm, mainloop_thread
except NameError:
    wm = newm.layout.Layout(debug=debug, config_file=config_file)
    newm.layout.Layout._INSTANCE = wm

    # Unlike run.py, launch main loop in a sub-thread so this notebook remains responsive.
    # `Layout` does spawn `LayoutThread` AND (via PyWM constructor) `PyWMThread` instances,
    # but the inherited `PyWM.run` method blocks calls wm.c `run()` which runs wlroots main loop...
    # This all makes naming tricky :-D
    mainloop_thread = threading.Thread(name='mainloop', target=wm.run)
    mainloop_thread.start()


NameError: name 'newm' is not defined

In [11]:
import time; time.sleep(5)

In [12]:
Cleanup
#newm.config.load_config(path_str=config_file)
wm.update_config()
# config.py gets imported (after setting up sys.path) as regular python module.
import config
config.NESTED

NameError: name 'wm' is not defined

- [ ] TODO: embed config in notebook and just re-execute those cells to "reload" it?  Could be nice "one workflow to rule them all"

In [13]:
wm

NameError: name 'wm' is not defined

In [14]:
while not wm.layout:  # Wait until Layout thread initializes
    print('.', end='')
    time.sleep(1)
wm.layout  # The outputs

NameError: name 'wm' is not defined

In [15]:
wm.workspaces

NameError: name 'wm' is not defined

In [16]:
print(wm.debug_str())

NameError: name 'wm' is not defined

In [17]:
wm._views

NameError: name 'wm' is not defined

In [18]:
[v] = [v for v in wm.views() if 'NEWM.ipynb' in v.title]
v_state, ws_state, v_id = wm.state.find_view(v)
v_state, ws_state, v_id

NameError: name 'wm' is not defined

In [19]:
print(wm.to_text(wm.workspaces[0]))

NameError: name 'wm' is not defined

In [20]:
wm.state.get_workspace_state(wm.workspaces[0]).size

NameError: name 'wm' is not defined

In [21]:
wm.basic_scale(1)

NameError: name 'wm' is not defined

In [22]:
wm.basic_scale(-1)

NameError: name 'wm' is not defined

In [23]:
wm.focus_view(v)

NameError: name 'wm' is not defined

In [24]:
wm.move_focused_view(-1, 1)

NameError: name 'wm' is not defined

In [25]:
#wm.terminate()
mainloop_thread

NameError: name 'mainloop_thread' is not defined